**1) Предсказать стоимость дома price - метрика MAPE**
2) При предсказании не должны использоваться шумящие столбцы. 
3) Внимание! Вывод ИМЕН перечней столбцов использованых в предсказании обязателен

### Подключение библиотек

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, ShuffleSplit, KFold, StratifiedShuffleSplit
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
from sklearn.preprocessing import StandardScaler, PolynomialFeatures, OneHotEncoder
from sklearn.pipeline import make_pipeline, Pipeline

from sklearn.compose import ColumnTransformer
import category_encoders as ce
from sklearn import svm

from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.compose import ColumnTransformer

### Обучение и выполнение лучшей модели

In [2]:
# Путь к тренировочному набору
path_train = 'train_house_A.csv' # содержит только имя файла, без имен папок

In [3]:
df = pd.read_csv(path_train)
df.shape

(408, 30)

In [4]:
df.head()

,price,0,1,2,3,4,5,6,7,8,...,19,20,21,22,23,24,25,26,27,28
0,5320000,0.405623,0.0,-0.061169,-0.741051,-0.223185,1.361397,1.221991,-0.570187,-0.478408,...,0.245475,-0.263991,0.047278,1.222962,-0.219265,1.804941,-0.126718,0.271470,1.391006,-0.141322
1,3465000,0.405623,0.0,-0.826890,-0.867066,-0.223185,-0.734539,-0.964205,-0.570187,-0.478408,...,-1.444928,-0.263991,0.047278,1.222962,-0.219265,-0.554035,-0.126718,-1.010746,0.617421,-0.141322
2,3500000,0.405623,0.0,-0.963310,-1.676106,-0.223185,-0.734539,-0.692084,-0.570187,-0.478408,...,1.578828,-0.263991,0.047278,1.222962,-0.219265,-0.554035,-0.126718,-1.237596,1.483005,-0.141322
3,3570000,0.405623,0.0,0.242611,0.193424,-0.223185,-0.734539,-0.696696,-0.570187,1.307431,...,-1.675466,-0.263991,-1.308863,1.222962,-0.219265,-0.554035,0.417348,-0.987324,1.134657,-0.141322
4,4130000,0.405623,0.0,-0.740827,-0.684767,-0.223185,-0.734539,-0.239163,-0.570187,-0.478408,...,-1.347578,-0.263991,1.403419,-0.091662,-0.219265,-0.554035,-0.108582,0.838006,0.360062,-0.141322


In [5]:
df.describe()

,price,0,1,2,3,4,5,6,7,8,...,19,20,21,22,23,24,25,26,27,28
count,4.080000e+02,408.000000,408.0,408.000000,408.000000,408.000000,408.000000,408.000000,408.000000,408.000000,...,408.000000,408.000000,408.000000,408.000000,408.000000,408.000000,408.000000,408.000000,408.000000,408.000000
mean,4.643597e+06,-0.030652,0.0,-0.021898,-0.004470,-0.044734,-0.015345,-0.037179,0.005930,-0.005686,...,-0.046552,-0.022184,-0.042466,0.017890,-0.043530,-0.039454,-0.038181,-0.048110,-0.025252,-0.044926
std,1.711368e+06,1.031893,0.0,1.019428,0.974612,0.969756,0.996282,1.001595,0.976837,1.007006,...,0.991383,1.020601,0.995270,1.016147,0.900616,0.975409,1.031259,1.008422,1.001805,0.913917
min,1.750000e+06,-2.465344,0.0,-1.224688,-1.780233,-2.633407,-0.734539,-1.614530,-0.570187,-4.050087,...,-1.705711,-2.319351,-2.665004,-1.406286,-0.219265,-0.554035,-5.000645,-1.789595,-1.705457,-3.418780
25%,3.430000e+06,0.405623,0.0,-0.673496,-0.837351,-0.223185,-0.734539,-0.745125,-0.570187,-0.478408,...,-0.881357,-0.263991,-1.308863,-1.406286,-0.219265,-0.554035,-0.126718,-0.896013,-0.887341,-0.141322
50%,4.270000e+06,0.405623,0.0,-0.252391,0.032263,-0.223185,-0.734539,-0.300045,-0.570187,-0.478408,...,-0.072463,-0.263991,0.047278,-0.091662,-0.219265,-0.554035,-0.126718,-0.047954,-0.050893,-0.141322
75%,5.600000e+06,0.405623,0.0,0.312075,0.775898,-0.223185,1.361397,0.454055,1.421812,1.307431,...,0.755884,-0.263991,0.047278,1.222962,-0.219265,-0.554035,-0.006287,0.787204,0.798591,-0.141322
max,1.141000e+07,0.405623,0.0,8.604753,1.732550,13.826353,1.361397,5.096263,3.413810,3.093270,...,1.745647,7.957450,4.115701,1.222962,4.560702,1.804941,13.021551,1.732996,1.771548,3.136137


In [6]:
df = df.drop_duplicates()
df.shape

(408, 30)

In [7]:
a = df.corr()

In [8]:
a[df.corr()>0.3]

,price,0,1,2,3,4,5,6,7,8,...,19,20,21,22,23,24,25,26,27,28
price,1.000000,0.300066,NaN,NaN,NaN,0.428015,NaN,0.512921,0.466621,NaN,...,NaN,NaN,0.360709,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,0.300066,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.428015,NaN,NaN,NaN,NaN,1.000000,NaN,0.488743,0.355175,NaN,...,NaN,0.436746,NaN,NaN,NaN,NaN,0.626080,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,0.512921,NaN,NaN,NaN,NaN,0.488743,NaN,1.000000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,0.466621,NaN,NaN,NaN,NaN,0.355175,NaN,NaN,1.000000,NaN,...,NaN,0.465341,0.381012,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
b=df.corr().drop('price', axis=1)
b=b.drop('price', axis=0)
b[b>0.3].sum()

0     1.000000
1     0.000000
2     1.000000
3     1.000000
4     2.906744
5     1.355768
6     1.821288
7     2.508427
8     1.000000
9     1.355768
10    2.300604
11    1.000000
12    2.889431
13    1.332545
14    1.000000
15    1.000000
16    1.000000
17    1.000000
18    1.000000
19    1.000000
20    2.902417
21    2.382433
22    1.000000
23    1.000000
24    1.000000
25    2.367804
26    1.000000
27    1.000000
28    2.141166
dtype: float64

In [10]:
features = a[df.corr()>0.3]['price'].dropna().index

In [11]:
features

Index(['price', '0', '4', '6', '7', '12', '13', '16', '21'], dtype='object')

In [12]:
data=df[features]
data

,price,0,4,6,7,12,13,16,21
0,5320000,0.405623,-0.223185,1.221991,-0.570187,-0.929397,1.517692,1.472618,0.047278
1,3465000,0.405623,-0.223185,-0.964205,-0.570187,-0.929397,-0.805741,-0.679063,0.047278
2,3500000,0.405623,-0.223185,-0.692084,-0.570187,0.224410,-0.805741,-0.679063,0.047278
3,3570000,0.405623,-0.223185,-0.696696,-0.570187,-0.929397,-0.805741,-0.679063,-1.308863
4,4130000,0.405623,-0.223185,-0.239163,-0.570187,0.224410,-0.805741,1.472618,1.403419
...,...,...,...,...,...,...,...,...,...
403,4550000,0.405623,-0.223185,-0.696696,-0.570187,0.224410,-0.805741,1.472618,0.047278
404,5250000,0.405623,-0.223185,-0.622901,-0.570187,0.224410,0.355976,-0.679063,0.047278
405,5950000,0.405623,-0.223185,0.640850,-0.570187,-0.929397,1.517692,-0.679063,0.047278
406,4319000,0.405623,-0.223185,-0.515897,-0.570187,-0.929397,-0.805741,-0.679063,-1.308863


In [13]:
y = (data.price)
X = data.drop(columns=['price'])

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [15]:
data.columns

Index(['price', '0', '4', '6', '7', '12', '13', '16', '21'], dtype='object')

In [16]:
poly_transformer = Pipeline(steps=[
    ('polynom', PolynomialFeatures(3,include_bias=False))])

CT = ColumnTransformer([
    ('num_transform', StandardScaler(with_mean=False), X.columns),
    ('poly_transform', poly_transformer, X.columns)
])

display(CT)
X_train_ct = CT.fit_transform(X_train)
X_test_ct = CT.transform(X_test)

ColumnTransformer(transformers=[('num_transform',
                                 StandardScaler(with_mean=False),
                                 Index(['0', '4', '6', '7', '12', '13', '16', '21'], dtype='object')),
                                ('poly_transform',
                                 Pipeline(steps=[('polynom',
                                                  PolynomialFeatures(degree=3,
                                                                     include_bias=False))]),
                                 Index(['0', '4', '6', '7', '12', '13', '16', '21'], dtype='object'))])

In [17]:
parameters = [
    {
        'regressor': [LinearRegression()]
    },
    {
        'regressor': [DecisionTreeRegressor()],
        'regressor__criterion':['squared_error','absolute_error'],
        'regressor__min_samples_leaf':[1,2,3]
    },
    {
        'regressor': [KNeighborsRegressor()],
#         'regressor__p':[1,2,3]
    },
    {'regressor':[GradientBoostingRegressor()]
    },
    {
     'regressor':[RandomForestRegressor()]
    },
]
pipe = Pipeline([('regressor', KNeighborsRegressor())])

In [18]:
grid = GridSearchCV(pipe, parameters, return_train_score=True, error_score='raise', scoring='neg_mean_absolute_percentage_error')
grid.fit(X_train, y_train)    

GridSearchCV(error_score='raise',
             estimator=Pipeline(steps=[('regressor', KNeighborsRegressor())]),
             param_grid=[{'regressor': [LinearRegression()]},
                         {'regressor': [DecisionTreeRegressor()],
                          'regressor__criterion': ['squared_error',
                                                   'absolute_error'],
                          'regressor__min_samples_leaf': [1, 2, 3]},
                         {'regressor': [KNeighborsRegressor()]},
                         {'regressor': [GradientBoostingRegressor()]},
                         {'regressor': [RandomForestRegressor()]}],
             return_train_score=True,
             scoring='neg_mean_absolute_percentage_error')

In [19]:
grid_result = pd.DataFrame(grid.cv_results_).sort_values(["rank_test_score",'std_test_score']).T
grid_result

,0,8,7,9,3,6,5,2,4,1
mean_fit_time,0.004071,0.048202,0.002432,0.127808,0.003357,0.004271,0.004766,0.003261,0.005617,0.002778
std_fit_time,0.002827,0.000799,0.000638,0.009494,0.001493,0.00013,0.000238,0.001576,0.000232,0.000422
mean_score_time,0.001977,0.001581,0.002772,0.009051,0.002018,0.001137,0.001218,0.00243,0.001381,0.001697
std_score_time,0.000439,0.00004,0.001022,0.001178,0.000908,0.000003,0.000009,0.001333,0.000142,0.000209
param_regressor,LinearRegression(),GradientBoostingRegressor(),KNeighborsRegressor(),RandomForestRegressor(),DecisionTreeRegressor(),DecisionTreeRegressor(),DecisionTreeRegressor(),DecisionTreeRegressor(),DecisionTreeRegressor(),DecisionTreeRegressor()
param_regressor__criterion,NaN,NaN,NaN,NaN,squared_error,absolute_error,absolute_error,squared_error,absolute_error,squared_error
param_regressor__min_samples_leaf,NaN,NaN,NaN,NaN,3,3,2,2,1,1
params,{'regressor': LinearRegression()},{'regressor': GradientBoostingRegressor()},{'regressor': KNeighborsRegressor()},{'regressor': RandomForestRegressor()},"{'regressor': DecisionTreeRegressor(), 'regres...","{'regressor': DecisionTreeRegressor(), 'regres...","{'regressor': DecisionTreeRegressor(), 'regres...","{'regressor': DecisionTreeRegressor(), 'regres...","{'regressor': DecisionTreeRegressor(), 'regres...","{'regressor': DecisionTreeRegressor(), 'regres..."
split0_test_score,-0.198918,-0.205557,-0.211759,-0.21914,-0.245876,-0.229429,-0.245638,-0.271468,-0.273033,-0.274736
split1_test_score,-0.20215,-0.217408,-0.234976,-0.25486,-0.288346,-0.32054,-0.326272,-0.328243,-0.391181,-0.36351


In [20]:
pipe = GradientBoostingRegressor(learning_rate=0.01,n_estimators=200,random_state=42, subsample=0.8,
                                    max_depth = 4)
pipe.fit(X_train_ct,y_train)
y_predict_best = pipe.predict(X_test_ct)

print('Оценка на тестовом наборе: ', mean_absolute_percentage_error(y_predict_best, y_test))

Оценка на тестовом наборе:  0.18392152726308034


In [21]:
gbs = GradientBoostingRegressor(learning_rate=0.01,n_estimators=200,random_state=42, subsample=0.8,
                                     max_depth = 4)
gbs.fit(X_train_ct,y_train)
importances = gbs.feature_importances_
slices = np.argsort(importances)[-22:]
slices

array([ 70, 156, 131, 118, 111, 132, 129, 147,  54, 110,  73, 122, 119,
       169, 159,  85, 106, 113, 134,  89, 136,  96])

In [22]:
final = GradientBoostingRegressor(learning_rate=0.01,n_estimators=200,random_state=42, subsample=0.8,
                                     max_depth = 4)
final.fit(CT.transform(X_train)[:, slices], y_train)
mean_absolute_percentage_error(y_test, final.predict(CT.transform(X_test)[:, slices]))

0.19912766742832366

In [23]:
X_train.columns

Index(['0', '4', '6', '7', '12', '13', '16', '21'], dtype='object')

# Итоговая модель

In [24]:
a = df.corr()
a[df.corr()>0.3]
features = a[df.corr()>0.3]['price'].dropna().index
features
data=df[features]

In [25]:
y = (data.price)
X = data.drop(columns=['price'])

In [26]:
path_test = 'train_house_A.csv'
test = pd.read_csv(path_test)

In [27]:
test = test[X.columns]

In [28]:
test.head()

,0,4,6,7,12,13,16,21
0,0.405623,-0.223185,1.221991,-0.570187,-0.929397,1.517692,1.472618,0.047278
1,0.405623,-0.223185,-0.964205,-0.570187,-0.929397,-0.805741,-0.679063,0.047278
2,0.405623,-0.223185,-0.692084,-0.570187,0.224410,-0.805741,-0.679063,0.047278
3,0.405623,-0.223185,-0.696696,-0.570187,-0.929397,-0.805741,-0.679063,-1.308863
4,0.405623,-0.223185,-0.239163,-0.570187,0.224410,-0.805741,1.472618,1.403419


In [29]:
poly_transformer = Pipeline(steps=[
    ('polynom', PolynomialFeatures(3,include_bias=False))])

CT = ColumnTransformer([
    ('num_transform', StandardScaler(with_mean=False), X.columns),
    ('poly_transform', poly_transformer, X.columns)
])

display(CT)
X_train_ct = CT.fit_transform(X)
test_ct = CT.transform(test)

ColumnTransformer(transformers=[('num_transform',
                                 StandardScaler(with_mean=False),
                                 Index(['0', '4', '6', '7', '12', '13', '16', '21'], dtype='object')),
                                ('poly_transform',
                                 Pipeline(steps=[('polynom',
                                                  PolynomialFeatures(degree=3,
                                                                     include_bias=False))]),
                                 Index(['0', '4', '6', '7', '12', '13', '16', '21'], dtype='object'))])

In [30]:
final = GradientBoostingRegressor(learning_rate=0.01,n_estimators=200,random_state=42, subsample=0.8,
                                     max_depth = 4)
final.fit(X_train_ct[:, slices], y)
y_predict = final.predict(test_ct[:, slices])

In [31]:
y_predict

array([5861686.16967553, 3273211.35420543, 3792896.81421076,
       3422667.88389879, 4608013.71128625, 3357710.86813182,
       4445967.57324145, 4196594.95129874, 6801723.2023192 ,
       7159822.98880437, 3585649.60761114, 6626882.73309007,
       6242107.249883  , 3850172.26853598, 4660260.96139113,
       3598481.60101832, 3524886.04754589, 4637997.09909594,
       5187134.09759021, 4750528.65100885, 4191710.3163479 ,
       3418441.7648912 , 4541697.35869503, 4079639.08922007,
       5630067.51379224, 4243593.83228385, 2906892.65268702,
       3305025.36090103, 3889105.72396208, 4822862.63031914,
       3490807.77125751, 3461002.62243486, 7266084.25023459,
       4400570.56345637, 4245954.8084682 , 4058768.46283625,
       3726984.16920271, 3215288.71759288, 5969035.84932271,
       7553283.62370598, 5136252.11154543, 3456996.53504809,
       3568675.9006567 , 3772081.7943407 , 4377143.54839732,
       5569518.36901256, 3812404.0872608 , 3290420.54657733,
       5307949.54283101,

In [34]:
from sklearn.metrics import mean_absolute_percentage_error
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt

y_true = pd.read_csv("test_Y_true_house_A.csv").values.reshape(-1)
print(y_predict.shape, y_true.shape)
print('Отрицательные цены - ', sum(y_predict<0))

def plot_2_error(y_true, y_predicted):
    residuals = y_true - y_predicted
    max_error = max(residuals) if abs(max(residuals)) > abs(min(residuals)) else min(residuals)
    max_idx = list(residuals).index(max(residuals)) if abs(max(residuals)) > abs(min(residuals)) else list(residuals).index(min(residuals))
    max_true, max_pred = y_true[max_idx], y_predicted[max_idx]
    mape = mean_absolute_percentage_error(y_true, y_predict)
    print("Max Error:", "{:,.0f}".format(max_error))
    print(' ')
    print('----------------------')
    print("   MAPE Error:", "{:,.2f}".format(mape*100))
    print('----------------------')

    ## Plot predicted vs true
    fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(15,5))
    from statsmodels.graphics.api import abline_plot
    ax[0].scatter(y_predicted, y_true, color="black")
    abline_plot(intercept=0, slope=1, color="red", ax=ax[0])
    ax[0].vlines(x=max_pred, ymin=max_true, ymax=max_true-max_error, color='red', linestyle='--', alpha=0.7, label="max error")
    ax[0].grid(True)
    ax[0].set(xlabel="Predicted", ylabel="True", title="Predicted vs True")
    ax[0].legend()

    ## Plot predicted vs residuals
    ax[1].scatter(y_predicted, residuals, color="red")
    ax[1].vlines(x=max_pred, ymin=0, ymax=max_error, color='black', linestyle='--', alpha=0.7, label="max error")
    ax[1].grid(True)
    ax[1].set(xlabel="Predicted", ylabel="Residuals", title="Predicted vs Residuals")
    ax[1].hlines(y=0, xmin=np.min(y_predicted), xmax=np.max(y_predicted))
    ax[1].legend()
    plt.show()

plot_2_error(y_true, y_predict)

(408,) (137,)
Отрицательные цены -  0


ValueError: operands could not be broadcast together with shapes (137,) (408,) 

In [33]:
mask = X_train.columns

import pickle
with open('saved_dictionary_a.pkl', 'rb') as f:
    loaded_dict = pickle.load(f)

color_dict = pd.Series(loaded_dict).to_frame().reset_index().sort_values(by=0).reset_index(drop=True)
color = (color_dict['index'].isin(mask)).map({True: 'background-color: yellow', False: ''})
color_dict.style.apply(lambda s: color)

,index,0
0,1,1
1,16,airconditioning
2,6,area
3,4,area bathrooms
4,25,area bedrooms
5,10,area stories
6,5,basement
7,7,bathrooms
8,28,bathrooms stories
9,21,bedrooms
